In [54]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta


### Loading the Files 
First, load the files and take a brief look at file contents and description. 

There are two main files : Database CPO (in GT GDrive), which contains updated information on all the CPOs and their respective contracts with GT which is updated by the commercial team. The second is the charge events excel file (download in admin.virta.fr -> download charge events (CPO)), which is a csv containing all the charge events for a give time period

note: in next versions, use scrapping to download and subtil to move to a folder

In [40]:

charge_events = pd.read_csv('cpochargeevents_01.08.2022 - 31.08.2022.csv',skiprows=1) #load the csv charge events file into pandas dataframe
charge_events.head() #see the first 5 columns of the csv file

,Created,Charge ID,Station ID,EVSE-ID,Station name,CPO name,Start time,Stop time,Duration,Energy (Wh),...,Free usage,Plug type,Charge method,Operator,Provider ID,Transaction ID,CPO share,Billing type,Location type,RFID
0,2022-08-16 15:22:22,28054331,45640,FR*EZD*E45640,Borne M. Joubert - 3kW,SURAYA,2022-08-16 15:22:22,2022-08-16 15:22:51,1,10,...,No,AC,auth_not_required_ocpp,FR*EZD,FI*VIR,26465122.0,0.0,Postpaid,Public,12345678
1,2022-08-25 23:03:49,28483424,45640,FR*EZD*E45640,Borne M. Joubert - 3kW,SURAYA,2022-08-25 23:03:49,2022-08-26 15:37:58,995,44195,...,No,AC,auth_not_required_ocpp,FR*EZD,FI*VIR,26860518.0,0.0,Postpaid,Public,12345678
2,2022-08-01 03:23:10,27306893,142517,FR*EZD*E142517,Created 2022-07-15 15:55 con: 0,SURAYA,2022-08-01 03:23:10,2022-08-01 14:58:22,696,11017,...,Yes,AC,rfid,FR*EZD,FI*VIR,25791463.0,0.0,Postpaid (by card),Public,123456789
3,2022-08-02 06:06:15,27358345,142517,FR*EZD*E142517,Created 2022-07-15 15:55 con: 0,SURAYA,2022-08-02 06:06:15,2022-08-02 15:29:41,564,4539,...,Yes,AC,rfid,FR*EZD,FI*VIR,25895168.0,0.0,Postpaid (by card),Public,123456789
4,2022-08-04 07:36:22,27456206,142517,FR*EZD*E142517,Created 2022-07-15 15:55 con: 0,SURAYA,2022-08-04 07:36:22,2022-08-04 15:49:02,493,7244,...,Yes,AC,rfid,FR*EZD,FI*VIR,25988570.0,0.0,Postpaid (by card),Public,123456789


In [42]:
charge_events.info() #see a description of the columns and data types 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5770 entries, 0 to 5769
Data columns (total 28 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Created         5770 non-null   object 
 1   Charge ID       5770 non-null   int64  
 2   Station ID      5770 non-null   int64  
 3   EVSE-ID         5770 non-null   object 
 4   Station name    5770 non-null   object 
 5   CPO name        5770 non-null   object 
 6   Start time      5770 non-null   object 
 7   Stop time       5767 non-null   object 
 8   Duration        5770 non-null   int64  
 9   Energy (Wh)     5770 non-null   int64  
 10  Meter start     5770 non-null   int64  
 11  Meter stop      5770 non-null   int64  
 12  VAT P           5770 non-null   float64
 13  VAT             5770 non-null   float64
 14  Net             5770 non-null   float64
 15  Price           5770 non-null   float64
 16  Currency        5760 non-null   object 
 17  Public/Private  5770 non-null   o

### Pre-treatment
note that non-nullcount difference between start time and stop time from charge_events.info() indicate the number of 'error charges' that have to be fixed. In other words, charges that were not registered correctly, and therefore do not have a corresponding stop time. This must be fixed before making the reports - all NaN stop values should be changed to 1 hour after the start time. To do this, first change the data type of time columns to datetime, then add 1 hour to all NaN cells in the 'Stop time' column.

In [43]:
time_24 = ['Created','Stop time','Start time']
charge_events[time_24] = charge_events[time_24].apply(pd.to_datetime,format = '%Y-%m-%d %H:%M:%S')



In [56]:
missing_stop_time_idx = np.where(charge_events['Stop time'].isnull())

In [60]:
charge_events['Stop time'].loc[missing_stop_time_idx] = charge_events['Start time'].loc[missing_stop_time_idx]+timedelta(hours=1) #adding 1 hour to NaN values


/home/stella/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [62]:
charge_events['Stop time'].loc[missing_stop_time_idx ]

251    2022-08-05 19:39:33
583    2022-08-03 19:40:43
1549   2022-08-24 16:44:15
Name: Stop time, dtype: datetime64[ns]

In [63]:
charge_events['Start time'].loc[missing_stop_time_idx ]

251    2022-08-05 18:39:33
583    2022-08-03 18:40:43
1549   2022-08-24 15:44:15
Name: Start time, dtype: datetime64[ns]

### CPO Groubys

In [65]:
charge_events.groupby(['CPO name']).apply(list)

CPO name
 Philippe JOCK       [Created, Charge ID, Station ID, EVSE-ID, Stat...
AFD                  [Created, Charge ID, Station ID, EVSE-ID, Stat...
AGSEA                [Created, Charge ID, Station ID, EVSE-ID, Stat...
ARS                  [Created, Charge ID, Station ID, EVSE-ID, Stat...
AUTOS GM             [Created, Charge ID, Station ID, EVSE-ID, Stat...
                                           ...                        
Solidarité SOS       [Created, Charge ID, Station ID, EVSE-ID, Stat...
Suprim Nuisibles     [Created, Charge ID, Station ID, EVSE-ID, Stat...
Taxi Thierry Lise    [Created, Charge ID, Station ID, EVSE-ID, Stat...
Unité Sud            [Created, Charge ID, Station ID, EVSE-ID, Stat...
VILDEUIL             [Created, Charge ID, Station ID, EVSE-ID, Stat...
Length: 125, dtype: object

In [69]:
test_CPO = charge_events.groupby(['CPO name']).get_group('AFD')
test_CPO

,Created,Charge ID,Station ID,EVSE-ID,Station name,CPO name,Start time,Stop time,Duration,Energy (Wh),...,Free usage,Plug type,Charge method,Operator,Provider ID,Transaction ID,CPO share,Billing type,Location type,RFID
3852,2022-08-31 19:04:54,28753500,38411,FR*EZD*E38411,AFD - Parking privé - 972 - 22kW,AFD,2022-08-31 19:04:54,2022-08-31 21:29:36,145,5668,...,Yes,AC,auth_not_required_ocpp,FR*EZD,FI*VIR,27062381.0,0.0,Prepaid,Office,12345678
3853,2022-08-31 21:30:10,28759376,38411,FR*EZD*E38411,AFD - Parking privé - 972 - 22kW,AFD,2022-08-31 21:30:10,2022-08-31 21:56:01,26,2821,...,Yes,AC,auth_not_required_ocpp,FR*EZD,FI*VIR,27063065.0,0.0,Prepaid,Office,12345678
3854,2022-08-31 22:51:19,28762261,38411,FR*EZD*E38411,AFD - Parking privé - 972 - 22kW,AFD,2022-08-31 22:51:19,2022-09-01 00:01:12,70,7687,...,Yes,AC,auth_not_required_ocpp,FR*EZD,FI*VIR,27065570.0,0.0,Prepaid,Office,12345678
